### Question 2: Gaussian Mixtures

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
import plotly.express as px

In [ ]:
# Load the data
data = np.load("temps.npz", allow_pickle=True)
X = data["X"]  # Temperature time series data (N, 366)
L = data["L"]  # Location data (Latitude, Longitude)
Xtr, Xte, Ltr, Lte = train_test_split(X, L, test_size=0.5, random_state=589)

print(f"Training set shape: {Xtr.shape}")
print(f"Test set shape: {Xte.shape}")
print(f"Location data shape: {Ltr.shape}")

### Question 2(a)

Learn the model on the training set using GaussianMixture with:
- covariance_type="diag"
- random_state=589
- max_iter=1000
- n_components=16

Then make a bar chart showing the learned mixture proportions.

In [ ]:
# Fit Gaussian Mixture Model with 16 clusters
gmm = GaussianMixture(
    n_components=16,
    covariance_type="diag",
    random_state=589,
    max_iter=1000
)
gmm.fit(Xtr)

# Extract the mixture proportions (weights)
mixture_proportions = gmm.weights_

print(f"Mixture proportions shape: {mixture_proportions.shape}")
print(f"Mixture proportions: {mixture_proportions}")
print(f"Sum of proportions: {mixture_proportions.sum()}")

In [ ]:
# Create bar chart of mixture proportions
plt.figure(figsize=(10, 6))
plt.bar(range(16), mixture_proportions, color='steelblue', edgecolor='black')
plt.xlabel('Cluster Index', fontsize=12)
plt.ylabel('Mixture Proportion', fontsize=12)
plt.title('Learned Mixture Proportions for 16-Component GMM', fontsize=14)
plt.xticks(range(16))
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('mixture_proportions.png', dpi=150, bbox_inches='tight')
plt.show()

### Question 2(b)

Visualize the parameters associated with each cluster:
- Each cluster has a mean vector of length 366 (daily mean temperature)
- Each cluster has a variance vector of length 366 (variability around mean)
- Create filled area plots showing 95% confidence intervals around the daily mean
- Overlay with line plots showing the daily mean temperature

In [ ]:
# Extract cluster parameters
means = gmm.means_  # Shape: (16, 366)
variances = gmm.covariances_  # Shape: (16, 366) for diagonal covariance

print(f"Means shape: {means.shape}")
print(f"Variances shape: {variances.shape}")

In [ ]:
# Create 4x4 grid of plots for all 16 clusters
fig, axes = plt.subplots(4, 4, figsize=(16, 12))
days = np.arange(1, 367)  # Day of year (1 to 366 for 2024 leap year)

# 95% confidence interval corresponds to approximately 1.96 standard deviations
z_score = 1.96

for k in range(16):
    row = k // 4
    col = k % 4
    ax = axes[row, col]
    
    mean_k = means[k]  # Mean temperature for cluster k
    std_k = np.sqrt(variances[k])  # Standard deviation for cluster k
    
    # Calculate 95% confidence bounds
    lower_bound = mean_k - z_score * std_k
    upper_bound = mean_k + z_score * std_k
    
    # Plot filled area for 95% confidence interval
    ax.fill_between(days, lower_bound, upper_bound, alpha=0.3, color='steelblue', label='95% CI')
    
    # Plot mean line
    ax.plot(days, mean_k, color='darkblue', linewidth=1.5, label='Mean')
    
    ax.set_title(f'Cluster {k}', fontsize=10)
    ax.set_xlabel('Day of Year', fontsize=8)
    ax.set_ylabel('Temperature', fontsize=8)
    ax.tick_params(axis='both', labelsize=7)
    ax.grid(alpha=0.3)
    
    # Add legend only to first plot
    if k == 0:
        ax.legend(fontsize=7, loc='upper right')

plt.suptitle('Daily Mean Temperature and 95% Confidence Interval by Cluster', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('cluster_parameters.png', dpi=150, bbox_inches='tight')
plt.show()

### Question 2(d)

Use the predict function to cluster the test data cases and plot each test weather station on a map colored by cluster.

In [ ]:
def plot_map(Z, L):
    """Plot locations on a map and color by cluster

    Parameters
    ----------
    Z : Array of cluster ids of shape (N,)
    L : Array of location Latitude, Longitude of shape (N,2)
    """
    # Latitude and Longitude Data
    data = {
        "Latitude": L[:, 0],
        "Longitude": L[:, 1],
        "Cluster": Z.astype(str)
    }

    # Create map using Plotly Express
    fig = px.scatter_geo(
        data, 
        lat="Latitude", 
        lon="Longitude", 
        color="Cluster",
        opacity=0.5,
        color_discrete_sequence=px.colors.qualitative.Prism,
        projection="robinson"  
    )

    fig.show()
    return fig

In [ ]:
# Predict cluster assignments for test data
Z_test = gmm.predict(Xte)

print(f"Test cluster assignments shape: {Z_test.shape}")
print(f"Unique clusters assigned: {np.unique(Z_test)}")
print(f"Cluster distribution in test set:")
for k in range(16):
    count = np.sum(Z_test == k)
    print(f"  Cluster {k}: {count} stations")

In [ ]:
# Plot the test weather stations on a map colored by cluster
fig = plot_map(Z_test, Lte)

# Optionally save the figure as an image (requires kaleido package)
# fig.write_image("cluster_map.png", scale=2)